<a href="https://colab.research.google.com/github/GiovanniSorice/Hate_Speech_Detection/blob/main/AlBerto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AlBERTo Hate Speech Classifier 

In [1]:
!pip install ekphrasis
!pip install tensorflow-gpu==1.15.0

import datetime
import sys
import warnings
warnings.filterwarnings("ignore")

#for code working
import tensorflow as tf
import re
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import numpy as np
import pandas as pd 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

#Prepare and import BERT modules
import subprocess
subprocess.call(["git", "clone", "https://github.com/google-research/bert","bert_repo"])

if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
from run_classifier import *
import modeling
import optimization
import tokenization

     |████████████████████████████████| 81kB 5.6MB/s 
     |████████████████████████████████| 184kB 13.0MB/s 
     |████████████████████████████████| 71kB 7.1MB/s 
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-cp37-none-any.whl size=82844 sha256=2d2dcd8afbd15387712e86b22f83a0e7f999941d0c5689b4a965ad7a1e3164d2
  Stored in directory: /root/.cache/pip/wheels/2f/c5/9b/c9b60f535a2cf9fdbc92d84c4801a010c35a9cd348011ed2a1
  Created wheel for ftfy: filename=ftfy-5.9-cp37-none-any.whl size=46451 sha256=129e14d213370658e9ca05d1204a7f950b70343979d442403501f5160324bdee
  Stored in directory: /root/.cache/pip/wheels/5e/2e/f0/b07196e8c929114998f0316894a61c752b63bfa3fdd50d2fc3
Successfully built ekphrasis ftfy
     |████████████████████████████████| 411.5MB 43kB/s 
     |████████████████████████████████| 3.8MB 47.6MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 512kB 51.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl 

In [2]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


## Load the dataset 

In [3]:
# directory name 
input_dir = '/content/drive/My Drive/HLT/clean_dataset_training/' 
AlBERTo_path = '/content/drive/MyDrive/HLT/alberto_uncased_L-12_H-768_A-12_italian_ckpt/'

# Spec
pd.set_option("display.max_colwidth", None)

In [4]:
tsv_file = open(input_dir+"training_dataset.csv")

dataset = pd.read_csv(tsv_file,sep=',')

# copy of tweets text 
tweets = dataset['text']  

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(dataset['text'], dataset['hs'], test_size=0.1)

## Configure AlBERTo classes 

In [6]:
text_processor = TextPreProcessor (
    # terms that will be normalized
    normalize=[ 'url' , 'email', 'user', 'percent', 'money', 'phone', 'time', 'date', 'number'] ,
    # terms that will be annotated
    annotate={"hashtag"} ,
    fix_html=True ,  # fix HTML tokens

    unpack_hashtags=True ,  # perform word segmentation on hashtags

    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    dicts = [ emoticons ]
)

Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading english - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/english/counts_1grams.txt
Reading english - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/english/counts_2grams.txt
Reading english - 1grams ...


# Prepare sentences to be converted 

In [7]:
i = 0 
examples_train = []
examples_test = []

for s in X_train: 
  examples_train.append([Y_train.iloc[i], s])
  i = i+1

i = 0 
for s in X_test: 
  examples_test.append([Y_test.iloc[i], s])
  i = i+1

examples_train = np.array(examples_train)
examples_test = np.array(examples_test)

In [8]:
'''
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create InputExample's using the constructor provided in the BERT library.

    text_a is the text we want to classify, which in this case, is the Request field in our Dataframe.
    text_b is used if we're training a model to understand the relationship between sentences (i.e. is text_b a translation of text_a? Is text_b an answer to the question asked by text_a?). This doesn't apply to our task, so we can leave text_b blank.
    label is the label for our example, i.e. True, False

'''
# guid = Globally unique ID for bookkeeping, unused in this example
f = lambda x: InputExample(guid=None, text_a = x[1], text_b = None, label = int(x[0]))

examples_train = map(f, examples_train)
examples_train = list(examples_train)

examples_test = map(f, examples_test)
examples_test = list(examples_test)

# Tokenizer 

In [9]:
VOCAB_FILE = '/content/drive/MyDrive/HLT/alberto_uncased_L-12_H-768_A-12_italian_ckpt/vocab.txt'

#Inizialize the tokenizer
tokenizer = tokenization.FullTokenizer(VOCAB_FILE, do_lower_case=True)

# Load Pre-Trained Model 

In [10]:
#model = AutoModel.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")

In [21]:
OUTPUT_DIR = '/content/drive/My Drive/HLT/output'
#Inizialize AlBERTo
INIT_CHECKPOINT = AlBERTo_path + 'alberto_model.ckpt'
#SET THE PARAMETERS
PREDICT_BATCH_SIZE = 1
MAX_SEQ_LENGTH = 128
LEARNING_RATE = 2e-5
label_list = [0, 1]
BERT_CONFIG= modeling.BertConfig.from_json_file(AlBERTo_path + "config.json")

#SET THE PARAMETERS FOR TRAINING 
TRAIN_BATCH_SIZE = 128
PREDICT_BATCH_SIZE = 1
EVAL_BATCH_SIZE = 512
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1.0
MAX_SEQ_LENGTH = 128
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500
 

In [18]:
#inizialize parameters
num_train_steps = 1  # int(len(X_train) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)+
num_warmup_steps = int(NUM_TRAIN_EPOCHS * WARMUP_PROPORTION) 
print(num_train_steps)
print(num_warmup_steps)

1
0


# Create model specification 

In [22]:
model_fn = model_fn_builder(
  bert_config=BERT_CONFIG,
  num_labels=2,
  init_checkpoint=INIT_CHECKPOINT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=False,
  use_one_hot_embeddings=True
)

## Setup run configuration 

In [23]:
def get_run_config():
  return tf.contrib.tpu.RunConfig(
    model_dir=AlBERTo_path)

In [24]:
estimator = tf.contrib.tpu.TPUEstimator(
  use_tpu=False,
  model_fn=model_fn,
  predict_batch_size=PREDICT_BATCH_SIZE,
  train_batch_size=TRAIN_BATCH_SIZE,
  config=get_run_config()
)

INFO:tensorflow:Using config: {'_model_dir': '/content/drive/MyDrive/HLT/alberto_uncased_L-12_H-768_A-12_italian_ckpt/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f050ef47e50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_p

# Fine-Tuning 

In [25]:
train_features = convert_examples_to_features(
      examples_train, label_list, MAX_SEQ_LENGTH, tokenizer)

train_input_fn = input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)

INFO:tensorflow:Writing example 0 of 6153
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: None
INFO:tensorflow:tokens: [CLS] accade e accaduto accadra ancora sono quattro anni che con ##batto per lo smantella ##ment ##del vamp ##o nomadi delle [SEP]
INFO:tensorflow:input_ids: 2 3487 13 5379 67390 52 37 879 115 15 35 93170 22 50 40575 8243 3859 38785 149 9058 119 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
print("***** Running training *****")
print("  Num examples = %d", len(examples_train))
print("  Num labels = %d", len(label_list))
print("  Batch size = %d", TRAIN_BATCH_SIZE)
print("  Num steps = %d", num_train_steps)

train_input_fn = input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)

print('***** Started training at {} *****'.format(datetime.datetime.now()))
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('***** Finished training at {} *****'.format(datetime.datetime.now()))

***** Running training *****
  Num examples = %d 6153
  Num labels = %d 2
  Batch size = %d 128
  Num steps = %d 1
***** Started training at 2021-03-08 10:54:48.710182 *****
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running train on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (128, 128)
INFO:tensorflow:  name = input_mask, shape = (128, 128)
INFO:tensorflow:  name = label_ids, shape = (128,)
INFO:tensorflow:  name = segment_ids, shape = (128, 128)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (128000, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, sha

# Prepare predictions

In [ ]:
# Setup sentences
input_features = convert_examples_to_features(
      examples_test, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn = input_fn_builder(
    features=input_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [ ]:
len(examples_test)

In [ ]:
predictions = estimator.predict(predict_input_fn)

for example, prediction in zip(tweets,predictions):
    print('\t prediction:%s \t text_a: %s' % ( np.argmax(prediction['probabilities']),str(example) ) )
#print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
#for example, prediction in zip(sentences,predictions):
 #   print('\t prediction:%s \t text_a: %s' % ( np.argmax(prediction['probabilities']),str(example) ) )